In [2]:
# !python make_dataset.py
# !python train_knn_model.py

In [3]:
from cascade.models import ModelRepo
from cascade.meta import MetricViewer
from cascade import data as cdd
from cascade import utils as cdu

In [4]:
from xaib.explainers.example_selection.constant_explainer import ConstantExplainer
from xaib.explainers.example_selection.random_explainer import RandomExplainer
from xaib.explainers.example_selection.knn_explainer import KNNExplainer

In [5]:
import os
import sys

sys.path.append(os.path.abspath('..'))
from utils import case

In [6]:
BS = 5

In [7]:
# Overwrite previous run
ModelRepo('repo', overwrite=True)

ModelRepo in repo of 0 lines

In [14]:
train_ds = cdd.Pickler('train_ds').ds()
test_ds = cdd.Pickler('test_ds')
n_features = train_ds.get_meta()[0]['n_features']
model = cdu.SkModel()
model.load('model')

In [15]:
explainers = {
    'const': ConstantExplainer(train_ds, train_ds[0]['item']),
    'random': RandomExplainer(train_ds),
    'shap': KNNExplainer(train_ds)
}

In [16]:
from xaib.cases.example_selection import ContinuityCase
from utils import NoiseApplier


MULTIPLIER = 0.01

@case(explainers=explainers, batch_size=BS)
def continuity() -> None:
    test_ds_noisy = NoiseApplier(test_ds, multiplier=MULTIPLIER)

    return ContinuityCase(test_ds, test_ds_noisy, model, multiplier=MULTIPLIER)

In [17]:
continuity()

20it [00:00, 3176.06it/s]
20it [00:00, 1926.56it/s]
20it [00:00, 343.91it/s]


In [19]:
repo = ModelRepo('repo')
t = MetricViewer(repo).table
t

,line,num,created_at,saved,small_noise_check,multiplier,name
0,repo\00000,0,2023-02-25T19:53:44.652541+00:00,a few seconds after,1.00,0.01,const
1,repo\00000,1,2023-02-25T19:53:44.652541+00:00,a few seconds after,0.00,0.01,random
2,repo\00000,2,2023-02-25T19:53:44.652541+00:00,a few seconds after,0.98,0.01,shap
